# Air Traffic Flow Management

This notebook contains the code to obtain data sets for the [Air Traffic Flow Management problem](https://doi.org/10.1016/j.cor.2019.104768) (ATFM) using flight plans corresponding to the US. The notebook is organized in different sections, and the details of the functions employed can be found in the corresponding `.jl` files.

The code was tested in [Julia](https://julialang.org/) 1.5.1, and the packages required are:
+ [Queryverse.jl](https://www.queryverse.org/)
+ [FreqTables.jl](https://github.com/nalimilan/FreqTables.jl)
+ [Distances.jl](https://github.com/JuliaStats/Distances.jl)
+ [LightGraphs.jl](https://github.com/JuliaGraphs/LightGraphs.jl)
+ [MetaGraphs.jl](https://github.com/JuliaGraphs/MetaGraphs.jl)

In [1]:
# Ignore the warnings (if any) when loading the packages
using Queryverse;
using FreqTables;
using Distances;
using LightGraphs;
using MetaGraphs;
using Statistics; # This pkg comes with Julia so it does not have to be installed

## Step 1: Raw data sets

### Step 1.1: Download and read data

We start downloading and reading the desired flight plan from [here](https://www.transtats.bts.gov/tables.asp?db_id=120&DB_Name=) and the location of the airports from [here](https://www.transtats.bts.gov/tables.asp?DB_ID=595&DB_Name=&DB_Short_Name=). Folder *./rawData* contains a case for both. Specifically, the flight information corresponds to January 2019. An explanation of the columns of the data sets can be found in the links provided.

In [2]:
include("functions_step1.jl");

In [3]:
DF_flights_raw  = DataFrame(load("./rawData/January2019.csv"));

In [4]:
DF_airports_raw = DataFrame(load("./rawData/airports.csv"));

**WARNING!!!**

Steps 1.2, 1.3 and 1.4 may cause an error if the name of the columns in the `csv` files is different than those used in `functions_step1.jl`. If you encounter this error, please modify accordingly the name of the columns in the data sets read. DO NOT modify the name in the code! Also note that you do not have to modify all the columns, just those used in `functions_step1.jl`.

### Step 1.2: Modify data referring to the flights

In the following, we clean `DF_flights_raw` by: 
1. Keeping only the flights of one selected day (check frequency table below).
2. Deleting canceled and diverted flights (flights that landed in a different airport than planned).
3. Selecting only the columns that we need. 
4. Deleting flights with missing information (NAs).
5. Keeping only territories of the contiguous US.
6. Deleting flights whose airport is not specified in the airport data set.

In [5]:
# To select the day of the flight plan (point 1 above), 
# one option is to  check the busiest days:
tbl =  freqtable(DF_flights_raw[:,:DayofMonth]);
sort(tbl, rev=true) # In this case is the 2nd day of the month

31-element Named Array{Int64,1}
Dim1  │ 
──────┼──────
2     │ 20384
11    │ 20082
25    │ 20041
7     │ 20015
18    │ 20009
10    │ 19980
24    │ 19963
31    │ 19962
17    │ 19960
14    │ 19941
28    │ 19934
4     │ 19566
⋮           ⋮
8     │ 18815
29    │ 18662
22    │ 18657
15    │ 18653
27    │ 18575
13    │ 18561
1     │ 18009
20    │ 16875
5     │ 16807
12    │ 15315
26    │ 15267
19    │ 14935

In [6]:
weekday = 25; # For this example, we select the 3rd busiest day
territoriesToDelte = ["Alaska",
                      "Hawaii",
                      "Puerto Rico",
                      "U.S. Pacific Trust Territories and Possessions",
                      "U.S. Virgin Islands"
                     ];

In [7]:
DF_flights = clean_data_flights(DF_flights_raw, weekday, territoriesToDelte, DF_airports_raw);

### Step 1.3: Modify data referring to the airports

For the airports: 
1. We select only those airports that are also in `DF_flights`, and the columns of interest to us.
2. We delete duplicities (we just want one pair of coordinates per airport).

In [8]:
DF_airports = clean_data_airports(DF_airports_raw, DF_flights);

### Step 1.4: Correcting wrong info in flight data

To conclude this first step we perform some extra modifications to the data frame with the flights. Specifically:
1. We transform the departure and arrival time information into a number between 0 and 24h\*60min/h = 1440.
2. In the data set, there are flights with the same tail number, but which are not continued flights (the departure and arrival airport do not match). We provide different tail numbers for those cases.
3. We add two columns to the data set. One to assign a flight number to each flight, and another to track which was the previous flight in a sequence of continued flights. In these sequences, the first flight is indicated with a -1.
4. In the data set employed (Jan/2019), we noticed that a few continued flights had a wrong time information: The arrival time of the previous flight occurred later than the departure time of the subsequent one. For those cases, we set the departure of the second flight at the arrival time of the first one + 30 minutes. 

In [9]:
modify_data_flights!(DF_flights);

In [10]:
# A couple of tests to check that the data make sense
test_df_flights(DF_flights)

## Step 2: Create Sectors and Route information

Once the raw data sets have been modified and corrected, we can start creating the sectors and route information. But first, we set the parameters needed in the rest of the code.

In [11]:
Base.@kwdef struct Input
    # Equivalence between minutes and periods
    I_periods::Int = 5; # 1 period = 5 minutes

    # SECTORS
    # number of columns and rows for the grid of the sectors
    I_numCol::Int = 20;
    I_numRow::Int = 20;

    # DEPARTURE & LANDING
    # Number of periods in departure and landing operations 
    I_periodDep::Int  = 1;
    I_periodLand::Int = 1;
    # Max departure delay. This is equal for all the flights.
    I_maxPeriodDelayDep::Int = ceil(Int, 1.5 * 60/I_periods); # 1:30 hours

    # FLYING
    # speed of the aircraft (km/h)
    D_speedAircraft::Float64 = 885;
    # % change of speed for delay and increase in air
    D_perDelay::Float64 = 0.25;
    D_perIncre::Float64 = 0.25;
    # Minimum s_{f,f'} time
    I_extraTime = 6;

    # ALTERNATIVES ROUTES
    # include alternative routes?
    B_altRoutes::Bool = true;
    # max number of alternative routes.
    I_maxNumRoutes::Int = 4;
    # percentage of sectors to check for alternative routes
    D_perSectors::Float64 = 0.05;
    # cost to penalize arcs and obtain alternative routes
    I_cost = 10_000;
end
input = Input();

In [12]:
include("functions_step2.jl");

The next function creates a data frame (`DF_graph`) with the arcs that connect:
+ The inner node of each sector with its boundary nodes (see details in the paper)
+ The airport with the boundary nodes.

The `DF_graph` data frame contains the following info/columns:
+ Number of arc.
+ Tail and head nodes.
+ Distance (cost) to go from one node to the other.
+ Sector to which the arc belongs.

Together with `DF_graph`, a dictionary (`dictAirportNode`) that assings to each AIRPORT_ID a node number is also created. Both will be needed in future steps.

In [13]:
DF_graph, dictAirportNode = create_sector_and_route_information!(input, DF_airports);

## Step 3: Create ATFM plans

Now it is time to create the ATFM plans. This basically means, given the flight information obtained in **Step 1** and the routes and sectors obtained in **Step 2**, combine them so a flight plan for each aircraft is obtained. That is, each aircraft ends up with a collection of potential (spatial) routes to use, as well as information about how much time it can spend crossing each sector that it encounters in its routes.

In [14]:
include("functions_step3.jl");

### Step 3.1: Preliminary data transformation

Transforming distance (i.e, cost) to travel time (in time periods)

In [15]:
κ = 1/input.D_speedAircraft * 60 * 1/input.I_periods; # time periods/km
DF_graph[!, :cost] = ceil.(Int, DF_graph[!, :cost] * κ); # ceil to guarantee a minimum value of 1

Creating a graph (instead of the data frame `DF_graph`)

In [16]:
numNodes = max(maximum(DF_graph[!, :tail]), maximum(DF_graph[!, :head]));
network  = MetaGraph(numNodes);
for r in eachrow(DF_graph)
    add_edge!(network, r[:tail], r[:head]);
    set_prop!(network, r[:tail], r[:head], :weight, r[:cost]);
    set_prop!(network, r[:tail], r[:head], :nArc, r[:nArc]);
    set_prop!(network, r[:tail], r[:head], :sector, r[:sector]);
end

### Step 3.2: Create ATFM plans for the main route

**Warning:** This step, as well as 3.3, may take some minutes.

In [17]:
# For each airport, we create a dummy node for departures and landings
dictAirportDummies = Dict{Int, Array{Int, 1}}();
counter = numNodes + 1;
for v in values(dictAirportNode)
    dictAirportDummies[v] = [counter, counter + 1];
    counter += 2;
end

# Put minutes into periods and move the time I_periodDep+1 units. This is to guarantee
# that no negative times will arise when the departure arcs are created.
DF_flights.DepTime = floor.(Int, DF_flights.DepTime/input.I_periods) .+ input.I_periodDep .+ 1;

In [18]:
DF_3droutes = create_main_3droutes(DF_flights, network, dictAirportNode, input, dictAirportDummies);

In `DF_3droutes`, each observation (row) is an arc connecting two nodes. For each arc, the following information is available (columns of `DF_3droutes`):
+ `flight`: Flight associated with the arc.
+ `net`: Aircraft that performs the flight. We use `net` as keyword due to the network structure that arises.
+ `seq`: Order in which the arcs forming the route from origin to destination should be traversed. Note that for each value of `flight` and `route` (see this later) the sequence is restarted.
+ `tail`: Tail node of the arc.
+ `head`: Head node of the arc.
+ `bt`: Preferred time at which the tail node should be left. We say "preferred" because due to some delay suffered in previous arcs of the route the actual departure time from a node can be different from the preferred one. 
+ `et`: Preferred time at which the head node should be reached. Note that `et` - `bt` is the preferred traversal time.
+ `delay`: Maximum delay allowed when traversing the arc. So if the tail node is left at time `t`, then it cannot reached the head node later than `t`+ preferred traversal time + `delay`. Note, that for arcs indicating the departure of an aircraft, `delay` means the maximum number of periods that the aircraft can be hold in ground.
+ `increase`: Maximum delay allowed when traversing the arc. So if the tail node is left at time `t`, then it cannot reached the head node sooner than `t`+ preferred traversal time - `increase`.
+ `route`: Number of the route. Value 1 indicates that the arc belong to the main/preferred route. Any other value indicates one the alternative routes available (if any).
+ `prevFlight`: Indicates the predecessor flight of the current flight. A value of -1 indicates that the flight has not predecessor. Note that this column is important to stablish connections between continued flights.
+ `phase`: Phase that represen the arc. Possible values are: dep, land, air; which stand for the 3 possible basic situations of an aircraft: departure, landing and flying.
+ `sector`: Sector employed by the arc when this is being used.
+ `cost`: Preferred traversal time, i.e, `et` - `bt`.

### Step 3.3: Create alternative routes

Notice that the alternative routes are only created if the boolean parameter in `input` is true

In [20]:
if input.B_altRoutes
    dictSectArcs = get_K_more_used_sectors(DF_3droutes, input, network,
                                            dictAirportNode);
    
    dictFlightSect = get_flights_using_busy_sectors(DF_3droutes, dictSectArcs,
                                                    dictAirportNode);
    
    DF_3dAlter = create_alternative_3droutes(DF_flights, network,
                                            dictAirportNode, input, dictAirportDummies,
                                            dictSectArcs, dictFlightSect);
    append!(DF_3droutes, DF_3dAlter);
end

### Step 3.4: Time connection between continued flights

Check if the minimum turnaround time (s_{f,f'}) is respected and if not, correct it.

In [21]:
sort!(DF_3droutes, [:flight, :route, :seq]);
check_time_in_continued_flights!(DF_3droutes, input);

## Step 4: Create capacity values

In [ ]:
Base.@kwdef struct RhsParameters
    baseValuesRule = :default;
    baseDifficulty = :easy; # easy, medium, difficult
    periodsOfWeatherPenalization::Int = 5;
end
parameters = RhsParameters();

In [12]:
# DF_3droutes = DataFrame(load("./3d.csv"));

In [42]:
using Statistics